In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

In [2]:
base_dir = "data/"

In [3]:
# kurupical
df1 = pd.read_csv(f"{base_dir}/model225/transformers1.csv")
df2 = pd.read_csv(f"{base_dir}/model226/transformers1.csv")
df3 = pd.read_csv(f"{base_dir}/model228/transformers1.csv")


In [4]:
# takoi-san
df_val_row = pd.read_feather("../input/riiid-test-answer-prediction/train_transformer_last2500k_only_row_id.feather")

def extract_val_row(path):
    df = pd.read_feather(path)
    return pd.merge(df, df_val_row, how="inner")
# takoi_san
df4 = extract_val_row(f"{base_dir}/takoi_ex55_56_57_lgb_9/transformer_55_56_57_lgb_9.feather")

In [5]:
# merge
df = pd.DataFrame()
df["target"] = df1["target"]
df["model225"] = df1["predict"]
df["model226"] = df2["predict"]
df["model228"] = df3["predict"]
df["ex55_56_57"] = df4["ex55"] + df4["ex56"] + df4["ex57"]
# df["ex55"] = df4["ex55"]
# df["ex56"] = df4["ex56"]
# df["ex57"] = df4["ex57"]

In [6]:
df_train = df.iloc[:2_000_000]
df_val = df.iloc[2_000_000:]

In [7]:
for col in ["model225", "model226", "model228", "ex55_56_57"]:
    score = roc_auc_score(df_val["target"], df_val[col])
    print(f"model={col} auc={round(score, 4)}")

model=model225 auc=0.8063
model=model226 auc=0.8073
model=model228 auc=0.8069
model=ex55_56_57 auc=0.8066


In [11]:
result = []

In [12]:
import random

In [14]:
for _ in range(100):
    ret = {}
    params = {
        'objective': 'binary',
        'max_depth': random.choice([2, 3, 4, 5, 6, -1]),
        'learning_rate': 0.01,
        'bagging_fraction': random.choice([0.5, 0.6, 0.7, 0.8]),
        'reg_alpha': random.choice([0, 0.1, 1, 10]),
        'reg_lambda': random.choice([0, 0.1, 1, 10]),
        'bagging_seed': 0,
        'random_state': 0,
        'metric': 'auc',
        'verbosity': -1,
        "n_estimators": 10000,
        "early_stopping_rounds": 50
   }
    print(params)
    # lgbm
    train_data = lgb.Dataset(df_train.drop("target", axis=1), label=df_train["target"])
    valid_data = lgb.Dataset(df_val.drop("target", axis=1), label=df_val["target"])

    model = lgb.train(
        params,
        train_data,
        valid_sets=[train_data, valid_data],
        verbose_eval=100
    )
    
    for k, v in params.items():
        ret[k] = v
    ret["auc"] = model.best_score["valid_1"]["auc"]

{'objective': 'binary', 'max_depth': -1, 'learning_rate': 0.01, 'bagging_fraction': 0.6, 'reg_alpha': 10, 'reg_lambda': 10, 'bagging_seed': 0, 'random_state': 0, 'metric': 'auc', 'verbosity': -1, 'n_estimators': 10000, 'early_stopping_rounds': 50}
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.813304	valid_1's auc: 0.810172
[200]	training's auc: 0.813444	valid_1's auc: 0.810319
[300]	training's auc: 0.813504	valid_1's auc: 0.81038
[400]	training's auc: 0.813552	valid_1's auc: 0.810416
[500]	training's auc: 0.813597	valid_1's auc: 0.810436
[600]	training's auc: 0.813638	valid_1's auc: 0.810448
[700]	training's auc: 0.813677	valid_1's auc: 0.810452
[800]	training's auc: 0.813704	valid_1's auc: 0.810456
[900]	training's auc: 0.813729	valid_1's auc: 0.810457
[1000]	training's auc: 0.813749	valid_1's auc: 0.810458
Early stopping, best iteration is:
[987]	training's auc: 0.813747	valid_1's auc: 0.810458
{'objective': 'binary', 'max_depth': 4, 'learning_r

[200]	training's auc: 0.813449	valid_1's auc: 0.810331
[300]	training's auc: 0.813511	valid_1's auc: 0.810397
[400]	training's auc: 0.813564	valid_1's auc: 0.810437
[500]	training's auc: 0.813615	valid_1's auc: 0.810456
[600]	training's auc: 0.813664	valid_1's auc: 0.810465
Early stopping, best iteration is:
[644]	training's auc: 0.813685	valid_1's auc: 0.810467
{'objective': 'binary', 'max_depth': 4, 'learning_rate': 0.01, 'bagging_fraction': 0.6, 'reg_alpha': 0, 'reg_lambda': 0, 'bagging_seed': 0, 'random_state': 0, 'metric': 'auc', 'verbosity': -1, 'n_estimators': 10000, 'early_stopping_rounds': 50}
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.813202	valid_1's auc: 0.810145
[200]	training's auc: 0.813322	valid_1's auc: 0.810268
[300]	training's auc: 0.813379	valid_1's auc: 0.810332
[400]	training's auc: 0.813414	valid_1's auc: 0.810374
[500]	training's auc: 0.81344	valid_1's auc: 0.810402
[600]	training's auc: 0.813465	valid_1's auc: 0.810421


[200]	training's auc: 0.813447	valid_1's auc: 0.810327
[300]	training's auc: 0.813507	valid_1's auc: 0.810391
[400]	training's auc: 0.813554	valid_1's auc: 0.810431
[500]	training's auc: 0.813592	valid_1's auc: 0.810452
[600]	training's auc: 0.813623	valid_1's auc: 0.810463
[700]	training's auc: 0.813648	valid_1's auc: 0.810466
[800]	training's auc: 0.813669	valid_1's auc: 0.810469
[900]	training's auc: 0.813686	valid_1's auc: 0.81047
Early stopping, best iteration is:
[873]	training's auc: 0.813682	valid_1's auc: 0.81047
{'objective': 'binary', 'max_depth': 2, 'learning_rate': 0.01, 'bagging_fraction': 0.7, 'reg_alpha': 0.1, 'reg_lambda': 1, 'bagging_seed': 0, 'random_state': 0, 'metric': 'auc', 'verbosity': -1, 'n_estimators': 10000, 'early_stopping_rounds': 50}
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.812366	valid_1's auc: 0.809405
[200]	training's auc: 0.813124	valid_1's auc: 0.810111
[300]	training's auc: 0.813297	valid_1's auc: 0.810274

[1300]	training's auc: 0.813563	valid_1's auc: 0.810452
[1400]	training's auc: 0.813576	valid_1's auc: 0.810455
[1500]	training's auc: 0.813592	valid_1's auc: 0.810458
[1600]	training's auc: 0.81361	valid_1's auc: 0.81046
[1700]	training's auc: 0.813634	valid_1's auc: 0.810463
Early stopping, best iteration is:
[1675]	training's auc: 0.813629	valid_1's auc: 0.810464
{'objective': 'binary', 'max_depth': 2, 'learning_rate': 0.01, 'bagging_fraction': 0.7, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'bagging_seed': 0, 'random_state': 0, 'metric': 'auc', 'verbosity': -1, 'n_estimators': 10000, 'early_stopping_rounds': 50}
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.812366	valid_1's auc: 0.809405
[200]	training's auc: 0.813124	valid_1's auc: 0.810111
[300]	training's auc: 0.813297	valid_1's auc: 0.810274
[400]	training's auc: 0.813345	valid_1's auc: 0.810329
[500]	training's auc: 0.813363	valid_1's auc: 0.810354
[600]	training's auc: 0.81337	valid_1's auc: 0

[1500]	training's auc: 0.813404	valid_1's auc: 0.810411
[1600]	training's auc: 0.81341	valid_1's auc: 0.810418
[1700]	training's auc: 0.813416	valid_1's auc: 0.810426
[1800]	training's auc: 0.813422	valid_1's auc: 0.81043
[1900]	training's auc: 0.813427	valid_1's auc: 0.810435
[2000]	training's auc: 0.813431	valid_1's auc: 0.810438
[2100]	training's auc: 0.813435	valid_1's auc: 0.810442
[2200]	training's auc: 0.81344	valid_1's auc: 0.810446
[2300]	training's auc: 0.813444	valid_1's auc: 0.810451
[2400]	training's auc: 0.813448	valid_1's auc: 0.810452
[2500]	training's auc: 0.813452	valid_1's auc: 0.810453
[2600]	training's auc: 0.813455	valid_1's auc: 0.810454
[2700]	training's auc: 0.813458	valid_1's auc: 0.810455
[2800]	training's auc: 0.813461	valid_1's auc: 0.810456
[2900]	training's auc: 0.813463	valid_1's auc: 0.810457
[3000]	training's auc: 0.813466	valid_1's auc: 0.810458
[3100]	training's auc: 0.813469	valid_1's auc: 0.81046
[3200]	training's auc: 0.81347	valid_1's auc: 0.8104

[300]	training's auc: 0.813516	valid_1's auc: 0.810395
[400]	training's auc: 0.813567	valid_1's auc: 0.810433
[500]	training's auc: 0.813611	valid_1's auc: 0.810453
[600]	training's auc: 0.813653	valid_1's auc: 0.810463
[700]	training's auc: 0.813693	valid_1's auc: 0.810465
Early stopping, best iteration is:
[696]	training's auc: 0.813692	valid_1's auc: 0.810466
{'objective': 'binary', 'max_depth': -1, 'learning_rate': 0.01, 'bagging_fraction': 0.6, 'reg_alpha': 0.1, 'reg_lambda': 0, 'bagging_seed': 0, 'random_state': 0, 'metric': 'auc', 'verbosity': -1, 'n_estimators': 10000, 'early_stopping_rounds': 50}
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.813311	valid_1's auc: 0.810182
[200]	training's auc: 0.813458	valid_1's auc: 0.810337
[300]	training's auc: 0.813524	valid_1's auc: 0.810404
[400]	training's auc: 0.81358	valid_1's auc: 0.810441
[500]	training's auc: 0.813635	valid_1's auc: 0.81046
[600]	training's auc: 0.813693	valid_1's auc: 0.81046

KeyboardInterrupt: 

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:156: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.813062	valid_1's auc: 0.810017
[200]	training's auc: 0.813279	valid_1's auc: 0.810242
[300]	training's auc: 0.813335	valid_1's auc: 0.810308
[400]	training's auc: 0.813358	valid_1's auc: 0.810341
[500]	training's auc: 0.813377	valid_1's auc: 0.810364
[600]	training's auc: 0.813396	valid_1's auc: 0.810388
[700]	training's auc: 0.813416	valid_1's auc: 0.810406
[800]	training's auc: 0.813429	valid_1's auc: 0.810418
[900]	training's auc: 0.813438	valid_1's auc: 0.810426
[1000]	training's auc: 0.813447	valid_1's auc: 0.810433
[1100]	training's auc: 0.813457	valid_1's auc: 0.81044
[1200]	training's auc: 0.813465	valid_1's auc: 0.810446
[1300]	training's auc: 0.813473	valid_1's auc: 0.81045
[1400]	training's auc: 0.813483	valid_1's auc: 0.810454
[1500]	training's auc: 0.813491	valid_1's auc: 0.810457
[1600]	training's auc: 0.813501	valid_1's auc: 0.81046
[1700]	training's auc: 0.813508	valid_1's auc: 0.810461

In [49]:
lgbm = model.predict(df_val.drop("target", axis=1))

In [50]:
from datetime import datetime as dt
import os
import pickle
now = dt.now().strftime("%Y%m%d%H%M%S")
os.makedirs(f"model/{now}")
with open(f"model/{now}/lgbm_stacking002.pickle", "wb") as f:
    pickle.dump(model, f)

# 以下、没になったモデリング集

In [94]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [93]:
from sklearn.ensemble import StackingClassifier

In [99]:
clf = StackingClassifier(
    estimators= [
        ('lgb', lgb.LGBMClassifier()),
        ('et', ExtraTreesClassifier()),
        ('rf', RandomForestClassifier()),
        ('lr', LogisticRegression()),
    ],
    final_estimator=LogisticRegression()
)

In [107]:
clf.fit(df_train.drop("target", axis=1).values, df_train["target"].values)

KeyboardInterrupt: 

In [ ]:
pred = clf.predict_proba(df_val.drop("target", axis=1).values)[:, 1]

In [78]:
# logistic reg
from sklearn.linear_model import LogisticRegression

clf_logistic = LogisticRegression()
clf_logistic.fit(df_train.drop("target", axis=1).values, df_train["target"].values)
logistic = clf_logistic.predict_proba(df_val.drop("target", axis=1).values)[:, 1]

In [79]:
roc_auc_score(df_val["target"].values, logistic)

0.8101269984455765

In [73]:
# mlp
from sklearn.neural_network import MLPClassifier

clf_mlp = MLPClassifier()
clf_mlp.fit(df_train.drop("target", axis=1).values, df_train["target"].values)
mlp = clf_mlp.predict(df_val.drop("target", axis=1).values)

In [81]:
mlp = clf_mlp.predict_proba(df_val.drop("target", axis=1).values)[:, 1]
roc_auc_score(df_val["target"].values, mlp)

0.8103880130787595

In [86]:
roc_auc_score(df_val["target"].values, lgbm*10 + logistic + mlp)

0.8104891955658622

In [54]:
import torch
import torch.nn.functional
import torch.utils.data


class Net(torch.nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 1)
    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [69]:
num_epochs = 100
X_train = torch.from_numpy(df_train.drop("target", axis=1).values).float().to("cuda")
y_train = torch.from_numpy(df_train["target"].values.reshape(-1, 1)).float().to("cuda")

X_val = torch.from_numpy(df_val.drop("target", axis=1).values).float().to("cuda")
y_val = df_val["target"].values.reshape(-1, 1)

clf_nn = Net(input_dim=6).to("cuda")
clf_nn.train()

# set training parameters
optimizer = torch.optim.Adam(clf_nn.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()

# start to train
epoch_loss = []
for epoch in range(num_epochs):
    # forward
    outputs = clf_nn(X_train)
    
    # calculate loss
    loss = criterion(outputs, y_train)
    
    # update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(roc_auc_score(y_val, clf_nn(X_val).data.cpu().numpy().tolist()))
    # save loss of this epoch

0.7909581340391222
0.7935409500374382
0.7956666729694192
0.7974297737518439
0.7988992712446684
0.8001329722162667
0.8011701916274742
0.8020340682563649
0.8027583226875239
0.8033708392113896
0.8038887175406277
0.804325117394177
0.8046982276649539
0.8050129981342328
0.8052841835169412
0.8055141863101432
0.8057102073651655
0.8058813882524619
0.8060306927844096
0.8061612273097309
0.8062760516807792
0.806376720136718
0.806466009344421
0.8065450104319479
0.8066167818319152
0.8066805997470996
0.8067388392407545
0.8067913341344606
0.8068394449512679
0.806883785839731
0.8069241703642148
0.8069614405929146
0.8069959036477212
0.8070278322825143
0.8070579214669422
0.8070863846635745
0.8071132274301601
0.807138871653504
0.807162856083742
0.8071856804767614
0.8072081889957349
0.8072291030381362
0.8072491589450804
0.8072681256371467
0.8072863242171664
0.8073029564383213
0.8073194043274748
0.8073341931333909
0.8073485380725576
0.8073623072479955
0.8073752850502173
0.8073876057914849
0.8073991259139354